# Additional simulated neural recordings from perturbed NA population

In [1]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import signal, stats
from sklearn import linear_model
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import get_seg, load_npy
from disp import set_font_size
from my_stats import nanpearsonr

from record_0_main import smlt_ma

cc = np.concatenate

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS

Load behavioral data from Coen et al 2014

In [2]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
df_behav

,ID,FRAME,T,Q,S,P,F,FFV,FFA,FLS,FRS,FMFV,FMLS
0,0,0,0.000000,1,0,0,0,0.231,0.0000,0.159,0.077,-0.058526,0.274257
1,0,1,0.033300,1,0,0,0,0.314,0.0385,0.025,0.044,0.100623,0.298490
2,0,2,0.066600,1,0,0,0,0.308,0.0025,0.029,0.011,0.147629,0.271865
3,0,3,0.099900,1,0,0,0,0.319,0.0020,0.016,0.016,0.091174,0.306112
4,0,4,0.133200,1,0,0,0,0.312,-0.0315,0.030,0.002,0.065871,0.306439
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4978565,275,27100,902.430902,1,0,0,0,0.217,0.0280,0.029,0.058,-0.171418,0.136184
4978566,275,27101,902.464202,1,0,0,0,0.210,-0.1045,0.138,0.322,0.032522,0.249171
4978567,275,27102,902.497502,1,0,0,0,0.008,-0.0910,0.034,0.260,0.033202,0.010844
4978568,275,27103,902.530803,1,0,0,0,0.028,0.0695,0.010,0.090,0.012059,0.027177


In [3]:
# split big df into dfs for individual trials
n_tr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in range(n_tr)]

tr_lens = np.array([len(df_tr) for df_tr in dfs_tr])
tr_lens_cum = cc([[0], np.cumsum(tr_lens)])

n_t_total = np.sum(tr_lens)
assert n_t_total == tr_lens_cum[-1]
df_behav = None

# Baseline MA model params

In [4]:
# neural response fits from Baker data
ma_param = load_npy('data/simple/neur/baker_ma.npy')

# Bounded response and adaptation timescales

In [5]:
KEYS_RTAU_RS_RTAU_AS = [
    ('medslow_r_fastmed_a', [2, 120], [.1, 20]),
]

In [6]:
for key, rtau_r, rtau_a in KEYS_RTAU_RS_RTAU_AS:
    print(key)
    SAVE_DIR = f'data/simple/mlv/ma_pert/baker_ma_{key}'
    SAVE_PFX = f'mlv_baker_ma_{key}'

    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
        
    ma_param = load_npy('data/simple/neur/baker_ma.npy')
    ma_param['TAU_R'] = np.random.uniform(*rtau_r, len(ma_param['TAU_R']))
    ma_param['TAU_A'] = np.random.uniform(*rtau_a, len(ma_param['TAU_A']))
    
    for ctr, df_tr in enumerate(dfs_tr):
        df_tr_nrl = df_tr.copy()
        sys.stdout.write('.')

        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)

        # simulate ppln response
        rs = smlt_ma(i_s, i_p, ma_param, DT)

        df_tr_nrl[[f'R_{expt_id}' for expt_id in ma_param['EXPT_ID']]] = rs

        np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

medslow_r_fastmed_a
....................................................................................................................................................................................................................................................................................